# <span style="color:gray">ipyrad-analysis toolkit:</span> RAxML



RAxML is the most popular tool for inferring phylogenetic trees using maximum likelihood. It is fast even for very large data sets. The documentation for raxml is huge, and there are many options. However, I tend to use the same small number of options very frequently, which motivated me to write the `ipa.raxml()` tool to automate the process of generating RAxml command line strings, running them, and accessing the resulting tree files. The simplicity of this tool makes it easy to incorporate into other more complex tools, for example, to infer tress in sliding windows along the genome using the `ipa.treeslider` tool.



### Required software

In [1]:
# conda install ipyrad -c bioconda
# conda install raxml -c bioconda
# conda install toytree -c conda-forge

In [2]:
import ipyrad.analysis as ipa
import toytree

### Short Tutorial:

The raxml tool takes a *phylip* formatted file as input. In addition you can set a number of analysis options either when you init the tool, or afterwards by accessing the `.params` dictionary. You can view the raxml command string that is generated from the input arguments and you can call `.run()` to start the tree inference. 
This example takes about 3-5 minutes to run on my laptop for a data set with 13 samples and ~1.2M SNPs and about 14% missing data.

In [6]:
# the path to your phylip formatted output file
phyfile = "../min10_outfiles/min10.phy"

In [7]:
# init raxml object with input data and (optional) parameter options
rax = ipa.raxml(data=phyfile, T=4, N=10)

# print the raxml command string for prosperity
print(rax.command)

# run the command, (options: block until finishes; overwrite existing)
rax.run(block=True, force=True)

raxmlHPC-PTHREADS-SSE3 -f a -T 4 -m GTRGAMMA -n test -w /home/deren/Documents/ipyrad/newdocs/cookbook/analysis-raxml -s /home/deren/Documents/ipyrad/tests/pedicularis/data10_outfiles/data10.phy -p 54321 -N 10 -x 12345
job test finished successfully


### Draw the inferred tree
After inferring a tree you can then visualize it in a notebook using `toytree`. 

In [8]:
# (optional) draw your tree in the notebook
import toytree

# load from the .trees attribute of the raxml object, or from the saved tree file
tre = toytree.tree(rax.trees.bipartitions)

# draw the tree
rtre = tre.root(wildcard="prz")
rtre.draw(tip_labels_align=True, node_labels="support");

<svg class="toyplot-canvas-Canvas" height="275.0px" id="ta5aa6ea46fbe4a25918efc9f28439d66" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 360.0 275.0" width="360.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 38362_rex 39618_rex 35236_rex 40578_rex 35855_rex 30556_thamno 33413_thamno 41954_cyathophylloides 41478_cyathophylloides 30686_cyathophylla 29154_superba 33588_przewalskii 32082_przewalskii 100 100 100 100 100 100 100 100 100 100 100 100

### Setting parameters

By default several parameters are pre-set in the raxml object. To remove those parameters from the command string you can set them to None. Additionally, you can build complex raxml command line strings by adding almost any parameter to the raxml object init, like below. You probably can't do everythin in raxml using this tool, it's only meant as a convenience. You can always of course just write the raxml command line string by hand instead.

In [23]:
# init raxml object
rax = ipa.raxml(data=phyfile, T=4, N=10)

# parameter dictionary for a raxml object
rax.params

N        10                  
T        4                   
binary   raxmlHPC-PTHREADS-SSE3
f        a                   
m        GTRGAMMA            
n        test                
p        54321               
s        ~/Documents/ipyrad/tests/pedicularis/data10_outfiles/data10.phy
w        ~/Documents/ipyrad/newdocs/cookbook/analysis-raxml
x        12345               

In [24]:
# paths to output files produced by raxml inference
rax.trees

bestTree                   ~/Documents/ipyrad/newdocs/cookbook/analysis-raxml/RAxML_bestTree.test
bipartitions               ~/Documents/ipyrad/newdocs/cookbook/analysis-raxml/RAxML_bipartitions.test
bipartitionsBranchLabels   ~/Documents/ipyrad/newdocs/cookbook/analysis-raxml/RAxML_bipartitionsBranchLabels.test
bootstrap                  ~/Documents/ipyrad/newdocs/cookbook/analysis-raxml/RAxML_bootstrap.test
info                       ~/Documents/ipyrad/newdocs/cookbook/analysis-raxml/RAxML_info.test

### Cookbook

Most frequently used: perform 100 rapid bootstrap analyses followed by 10 rapid hill-climbing ML searches from random starting trees under the GTRGAMMA substitution model. 

In [9]:
rax = ipa.raxml(
    data=phyfile,
    name="test",
    workdir="analysis-raxml",
    m="GTRGAMMA",
    T=20,
    f="a",
    N=100,
)
print(rax.command)

raxmlHPC-PTHREADS-SSE3 -f a -T 20 -m GTRGAMMA -n test -w /home/deren/Documents/ipyrad/newdocs/cookbook/analysis-raxml -s /home/deren/Documents/ipyrad/tests/pedicularis/data10_outfiles/data10.phy -p 54321 -N 100 -x 12345


Another common option: Perform N rapid hill-climbing ML analyses from random starting trees, with no bootstrap replicates.

In [10]:
rax = ipa.raxml(
    data=phyfile,
    name="test",
    workdir="analysis-raxml",
    m="GTRGAMMA",
    T=20,
    f="d",
    N=10,
    x=None,
)
print(rax.command)

raxmlHPC-PTHREADS-SSE3 -f d -T 20 -m GTRGAMMA -n test -w /home/deren/Documents/ipyrad/newdocs/cookbook/analysis-raxml -s /home/deren/Documents/ipyrad/tests/pedicularis/data10_outfiles/data10.phy -p 54321 -N 10


### Generating new phylip alignments
See the ipyrad-analysis `window_extracter` tool for creating new alignments by applying filtering or consensus calling to include/exclude taxa or loci/sites from analyses with options to minimize th amount of missing data in alignments. 

### What's next?

If you have reference mapped data then you should see the `.treeslider()` tool to infer trees in sliding windows along scaffolds; or the `.window_extracter()` tool to extract, filter, and concatenate RAD loci within a given window (e.g., near some known gene).